In [1]:
# Required Python packages
from datetime import date, datetime, timedelta
import calendar
import pandas as pd
import numpy as np
import itertools
from scipy.stats import norm
import copy
import os

#Import Date opertative user defined functions
from ipynb.fs.full.user_defined_vik_functions import find_last_thurs_date_of_month, date_of_prev_thurs, \
                                                     prev_weekday, prev_workday_if_holiday, \
                                                     next_working_day_incl_input_date, next_weekday, \
                                                     next_weekday_incl_input_date, \
                                                     get_all_monthly_option_expiries, find_wkly_expries, date_of_next_thurs \
                                                     

#Import pricing functions
from ipynb.fs.full.user_defined_vik_functions import BSM_call_with_div, BSM_put_with_div

#Import data loading functions
from ipynb.fs.full.user_defined_vik_functions import load_all_mthly_data, load_all_wkly_data_of_the_month, find_latest_available_mibor

# Import dataframe naming functions 
from ipynb.fs.full.user_defined_vik_functions import get_mthly_df_name_from_expiry

# Import risk free interest rate function
from ipynb.fs.full.user_defined_vik_functions import get_risk_free_rate_from_exact_date

#Path Settings
source_path = "/home/jupyter-partha/Vikranth - Chapter 2/"
input_sub_path = "Input Data/mkt_data_covid_region/"
output_sub_path = "Output Data/"
input_data_path = source_path + input_sub_path
output_data_path = source_path + output_sub_path


In [2]:
#Index of interest
# stock_ident = "BANKNIFTY"
stock_ident = "NIFTY"

# Periods of interest will be a dictionary
#Key is the year, value is a list of months 1-12, 1- Jan, 2 - Feb,...12 - Dec
# For E.g., periods_of_interest = {2020: [3], 2019: [11, 12]}

# periods_of_interest = {2019:[8, 9, 10, 11, 12], 2020: [1, 2, 3, 4, 5, 6, 7]}
periods_of_interest = {2020: [6, 7]}

#List of holidays
holidays_list = [date(2019, 3, 4), date(2019, 3, 21),\
                 date(2019, 4, 17), date(2019, 4, 19), date(2019, 4, 29),\
                 date(2019, 5, 1),\
                 date(2019, 6, 5),\
                 date(2019, 8, 12), date(2019, 8, 15),\
                 date(2019, 9, 2), date(2019, 9, 10), \
                 date(2019, 10, 2), date(2019, 10, 8), date(2019, 10, 21), date(2019, 10, 28), \
                 date(2019, 11, 12), \
                 date(2019, 12, 25), \
                 date(2020, 2, 21), \
                 date(2020, 3, 10), \
                 date(2020, 4, 2), date(2020, 4, 6), date(2020, 4, 10), date(2020,4, 14), \
                 date(2020, 5, 1), date(2020, 5, 25), \
                 date(2020, 10, 2), date(2020, 11, 16), date(2020, 11, 30), date(2020, 12, 25)]


In [3]:
# The following functions are user-defined functions used for 
# generating implied vol surface using Brent's Alogrithm

#Error Function - For European call and European put options
#Input: mkt_price of the option, option_type: "CE" for European Call, "PE" for European Put
#      (s_t, k, r, dt) -> (current stock price, strike, risk-free rate, time to maturity)
#Output: Error of model price in comparison with mkt price
def error_func(mkt_price, s_t, k, r, r_f, dt, option_type, iv):
    if (option_type == "CE"):
        model_price = BSM_call_with_div(s_t, k, r, r_f, iv, dt)
    else:
        model_price = BSM_put_with_div(s_t, k, r, r_f, iv, dt)
    error = model_price - mkt_price
    return error

#### O(n) Algorithm to find product of other elements ####
# Input: a list
#Output: A list with product of other elements except the element in the index
def product_other_elem(arr):
    n = len(arr)
    prod_arr = list(np.ones(n).astype(np.float64))
    
    temp = 1.0
    for i in range(n):
        prod_arr[i] = temp
        temp = temp * arr[i]
    temp = 1.0
    for i in range(n-1, -1, -1):
        prod_arr[i] = prod_arr[i] * temp
        temp = temp * arr[i]
    return prod_arr

#Generates the iv(k) as function of iv(k-1), iv(k-2), iv(k-3), mkt_price, BS price
#### Using Brent's Algorithm ####

#Inverse Quadratic interpolation
def inverse_quadratic_iv(mkt_price, s_t, k, r, r_f, dt, option_type, iv_list):
    g = lambda iv: error_func(mkt_price, s_t, k, r, r_f, dt, option_type, iv)
    g_k = [g(iv) for iv in iv_list]
    g_num = product_other_elem(g_k)
    g_den = [np.prod(g_k[i] - np.array(g_k[:i] + g_k[i+1:])) for i in range(0, len(iv_list))]
    next_iv = np.sum(np.array([(iv_list[i] * g_num[i]) / g_den[i] for i in range(0, len(iv_list))]))
    return next_iv

#Secant method
def secant_method_iv(mkt_price, s_t, k, r, r_f, dt, option_type, iv_list):
    g = lambda iv: error_func(mkt_price, s_t, k, r, r_f, dt, option_type, iv)
    g_k = [g(iv) for iv in iv_list]
    next_iv = (iv_list[1] - g_k[1]) * ((iv_list[1] - iv_list[0])/(g_k[1]-g_k[0]))
    return next_iv

#Bisection method
def bisection_method_iv(mkt_price, s_t, k, r, r_f, dt, option_type, iv_list):
    next_iv = (iv_list[0] + iv_list[1])/2
    return next_iv

#Swap the elements of 2 elements list
def swap(iv_list):
    temp = iv_list[1]
    iv_list[1] = iv_list[0]
    iv_list[0] = temp
    return iv_list

#Generate IV using Brent's algorithm
def gen_brents_iv(mkt_price, s_t, k, r, r_f, dt, option_type, iv_list_initial):
    iv_list = copy.deepcopy(iv_list_initial)
    g = lambda iv: error_func(mkt_price, s_t, k, r, r_f, dt, option_type, iv)    
    
    try:
        assert (g(iv_list[0]) * g(iv_list[1])) < 0
    except AssertionError:
        print("BSMVol0_price - mkt_price, BSMVol1_price - mkt_price: ", g(iv_list[0]), g(iv_list[1]))
        print("Trade_terms: s_t, k, dt, option_type, iv_list: (", s_t, k, dt, option_type, iv_list, ")")
        print("Mkt Price: ", mkt_price) 
        if (option_type == "CE"):
            print("IV0 BSM price: ", BSM_call_with_div(s_t, k, r, r_f, iv_list[0], dt))
            print("IV1 BSM price: ", BSM_call_with_div(s_t, k, r, r_f, iv_list[1], dt))
            
        else:
            print("IV0 BSM price: ", BSM_put_with_div(s_t, k, r, r_f, iv_list[0], dt))
            print("IV1 BSM price: ", BSM_put_with_div(s_t, k, r, r_f, iv_list[1], dt))
        print("The initialisations doesn't contain the solution within for interpolation")
    
    if (abs(iv_list[0] ) < abs(iv_list[1])):
        iv_list = swap(iv_list)

    mflag = 1
    
    opt_iv = iv_list[1]    
    num_iter = 0 #maximum iteration is set as 1000 in the while loop below
    eps = 0.0001 # 1 basis point is set as the maximum error allowed for vol

    
    while (abs(g(opt_iv)) >= 0.000001 and num_iter <=1000):   
        if (g(iv_list[0]) != g(iv_list[2]) and g(iv_list[1]) != g(iv_list[2])):
            #Inverse Quadratic Interpolation
            opt_iv = inverse_quadratic_iv(mkt_price, s_t, k, r, r_f, dt, option_type, iv_list)
        else:
            #Secant Method
            opt_iv = secant_method_iv(mkt_price, s_t, k, r, r_f, dt, option_type, iv_list)
        
        if ((( 3 * iv_list[0] + iv_list[1])/4 < opt_iv < iv_list[1]) \
            or ( mflag == 1 and (abs(opt_iv - iv_list[1]) >= abs(iv_list[1] - iv_list[2])/2 )) \
            or ( mflag == 0 and (abs(opt_iv - iv_list[1]) >= abs(iv_list[2] - prev_iv_2)/2 )) \
            or (mflag == 1 and (abs(iv_list[1] - iv_list[2]) < abs(eps))) \
            or (mflag == 0 and (abs(iv_list[2] - prev_iv_2) < abs(eps)))):
            
            opt_iv = (iv_list[0] + iv_list[1]) / 2
            mflag = 1
        else:
            mflag = 0
            

        prev_iv_2 = iv_list[2]
        iv_list[2] = iv_list[1]
        
        if (g(iv_list[0]) * g(opt_iv) < 0):
            iv_list[1] = opt_iv
        else:
            iv_list[0] = opt_iv
        
        if (abs(iv_list[0] ) < abs(iv_list[1])):
            iv_list = swap(iv_list)
        
        num_iter = num_iter + 1
    return opt_iv

#Function to generate Implied volatility Surface based on Brent's Algorithm
#Note: In vol surface: x-axis is moneyness ie. S/K for call options and K/S for put options
#                      y-axis is Time to maturity
#                      z-axis is implied volatility
#Inputs:
#       dt_list = [tenor1, tenor2....]
#       price_lists_tenorwise = [[Price for all options in tenor 1],[for tenor2],....]
#       strike_lists_tenorwise = [[Strike for all options in tenor 1],[for tenor2],....]
#       s_t - current stock price, r - risk free rate, option_type is "CE" or "PE"
#      initialisation of brent's iv is: [0.0000000001, 50, 50], 
#                 S.T, iv[0] creates a +ve error and iv[1] creates negative error,
#                where, error = (model price - mkt_price)
#Output:
#      A list of iv surface: [dict for tenor 1, dict for tenor 2,...]
#                      dict = {"Moneyness:" [], "T": [list of same value dt], "Impl_Vol": []}
def gen_brent_iv_surface(price_lists_tenorwise, s_t, strike_lists_tenorwise, r, r_f, dt_list, option_type, iv_list=[0.0000000001, 50, 50]):
    iv_surface = []
    for i in range(0, len(dt_list)):
        dt = dt_list[i]
        strike_list = strike_lists_tenorwise[i]
        price_list = price_lists_tenorwise[i]
        if(option_type == "CE"):
            moneyness_list = [s_t/strike for strike in strike_list]
        else:
            moneyness_list = [strike/s_t for strike in strike_list]
        dt_list_curr_tenor = [dt for evry_elm in price_list]
        spot_list = [s_t for evry_elm in price_list]
        impl_vol_list = [gen_brents_iv(price_list[i], s_t, strike_list[i], r, r_f, dt, option_type, iv_list) for i in range(0, len(price_list))]
        
        vol_smile_dict = {"Moneyness":moneyness_list,"T":dt_list_curr_tenor, "Spot": spot_list, "Strike":strike_list, "Impl_Vol":impl_vol_list}
        iv_surface.append(vol_smile_dict)
    return iv_surface

#Function to generate iv_surface dataframe from (iv-surface list -> list of iv_smile dictionaries)
def gen_iv_surf_dataframe(iv_surface_list_of_dict):
    list_of_df = [pd.DataFrame(iv_surf_dict) for iv_surf_dict in iv_surface_list_of_dict]
    list_of_df = [iv_df.sort_values('Moneyness') for iv_df in list_of_df]
    df_iv_surface = pd.concat(list_of_df, axis=0)
    return df_iv_surface


# We generate and store the volatility surface for 26th March expiry.
# Each day, we generate a volatility surface with time to maturity = 26th march - current date
# ACT/365 day count convention is used
#Input: periods_of =_interest as discussed above, holidays list, product type - "CE" or "PE"
#Output: List of vol surface dataframes and saved as csv in the data location
def gen_daily_vol_smiles_for_fixed_expiries(periods_of_interest, holidays_list, input_path, output_path, \
                                            stock_ident, prod_type = "CE"):
    expiries_of_interest = get_all_monthly_option_expiries(periods_of_interest, holidays_list)

    df_mkt = load_all_mthly_data(expiries_of_interest, input_path, \
                                 holidays_list, prod_type_lists=["CE", "PE"], stock_ident=stock_ident)

    df_futures_mkt = load_all_mthly_data(expiries_of_interest, input_path, \
                                         holidays_list, prod_type_lists=["FUT"], stock_ident=stock_ident)
    #Tricky nomenclature: following creates the for each month, first day of each week
    dict_wkly_expiries_each_month = find_wkly_expries(expiries_of_interest, holidays_list)
    mthly_list_of_vol_smiles = {}
    for expiry_date in expiries_of_interest:
        fixed_expiry_volsmiles_list = []
        if (expiry_date.month == 1):
            last_month = 12
            last_year = expiry_date.year - 1
        else:
            last_month = expiry_date.month - 1
            last_year = expiry_date.year 
            
        start_date = prev_workday_if_holiday(find_last_thurs_date_of_month(last_year, \
                                                                               last_month), \
                                                 holidays_list)

        curr_date =start_date
        df = df_mkt[get_mthly_df_name_from_expiry(expiry_date, prod_type, holidays_list, stock_ident)]
        
        # Condition to use only liquid options
        trd_volume_array = np.sort(np.array(df[df['No. of contracts'] > 0]['No. of contracts']))
        trd_volume_50th_pct = np.percentile(trd_volume_array, 50)
        oi_array= np.sort(np.array(df[df['Open Int'] > 0]['Open Int']))
        oi_50th_pct = np.percentile(oi_array, 50)
        
        df = df[df['Open Int'] > oi_50th_pct]
        df = df[df['No. of contracts'] > trd_volume_50th_pct]
        
        # To use settlement price
        if (prod_type == "CE"):
            df['Close'] = df.apply(lambda x: max(x['Underlying Value'] - x['Strike Price'], 0) if (x['Date'] == x['Expiry']) else \
                                   x['Settle Price'], axis=1) 
        else:
            df['Close'] = df.apply(lambda x: max(x['Strike Price'] - x['Underlying Value'], 0) if (x['Date'] == x['Expiry']) else \
                                   x['Settle Price'], axis=1) 
        
        df_futures = df_futures_mkt[get_mthly_df_name_from_expiry(expiry_date, "FUT", holidays_list, stock_ident)]
        
        wkly_expiries_list = dict_wkly_expiries_each_month[expiry_date.strftime("%d-%b-%Y")]
        
        dict_weekly_dfs = load_all_wkly_data_of_the_month(expiry_date, holidays_list, input_path, prod_type_lists=["CE", "PE"], stock_ident = stock_ident)
        
        week_id=0
        while(curr_date < expiry_date):
            
            if (curr_date == wkly_expiries_list[week_id]):
                wk_start = wkly_expiries_list[week_id]
                wk_end = prev_workday_if_holiday(date_of_next_thurs(wk_start), holidays_list)
                dt = float((expiry_date - curr_date).days) / 365
                dt_wk = float((wk_end - curr_date).days) / 365
                week_file_name = "OPTIDX_" + stock_ident + "_" + prod_type + "_" \
                        + wk_start.strftime("%d-%b-%Y") +  "_TO_" \
                        + wk_end.strftime("%d-%b-%Y") + ".csv"
                df_week = dict_weekly_dfs["df" + "_" + week_file_name[:-4]]
                df_wk = df_week[df_week['Date'] == curr_date.strftime("%d-%b-%Y")]
                if (df_wk.empty):
                    df_wk = df_week[df_week['Date'] == curr_date.strftime("%d-%b-%y")]
                
                # Condition to use only traded reasonably liquid options for vol calibration
                wk_trd_volume_array = np.sort(np.array(df_wk[df_wk['No. of contracts'] > 0]['No. of contracts']))
                wk_trd_volume_50th_pct = np.percentile(wk_trd_volume_array, 50)
                wk_oi_array= np.sort(np.array(df_wk[df_wk['Open Int'] > 0]['Open Int']))
                wk_oi_50th_pct = np.percentile(wk_oi_array, 50)
        
                df_wk = df_wk[df_wk['Open Int'] > wk_oi_50th_pct]
                df_wk = df_wk[df_wk['No. of contracts'] > wk_trd_volume_50th_pct]
                
                # To use settlement price
                if (prod_type == "CE"):
                    df_wk['Close'] = df_wk.apply(lambda x: max(x['Underlying Value'] - x['Strike Price'], 0) if (x['Date'] == x['Expiry']) else \
                                   x['Settle Price'], axis=1) 
                else:
                    df_wk['Close'] = df_wk.apply(lambda x: max(x['Strike Price'] - x['Underlying Value'], 0) if (x['Date'] == x['Expiry']) else \
                                   x['Settle Price'], axis=1) 
                
                df_curr = df[df['Date'] == curr_date.strftime("%d-%b-%Y")]
                if (df_curr.empty):
                    df_curr = df[df['Date'] == curr_date.strftime("%d-%b-%y")]

                s_t = float(df_curr['Underlying Value'].tolist()[0])
                
                if (week_id == len(wkly_expiries_list)-1):
                    price_lists_tenorwise = [df_curr['Close'].tolist()]
                    strike_lists_tenorwise = [df_curr['Strike Price'].tolist()]
                    dt_list = np.array([dt])
                
                else:
                    price_lists_tenorwise = [df_curr['Close'].tolist(), df_wk['Close'].tolist()]
                    strike_lists_tenorwise = [df_curr['Strike Price'].tolist(), df_wk['Strike Price'].tolist()]
                    dt_list = np.array([dt, dt_wk])
                
                
                week_id = week_id+1
                if (week_id >= len(wkly_expiries_list)):
                    week_id = 0
                
            else:
                dt = float((expiry_date - curr_date).days) / 365
                df_curr = df[df['Date'] == curr_date.strftime("%d-%b-%Y")]
                if (df_curr.empty):
                    df_curr = df[df['Date'] == curr_date.strftime("%d-%b-%y")]
                s_t = float(df_curr['Underlying Value'].tolist()[0])
                
                price_lists_tenorwise = [df_curr['Close'].tolist()]
                strike_lists_tenorwise = [df_curr['Strike Price'].tolist()]
                dt_list = np.array([dt])

            # We imply index returns (risk free rate -  dividend) from futures of same maturity 
            df_curr_futures = df_futures[df_futures['Date'] == curr_date.strftime("%d-%b-%Y")]
            if (df_curr_futures.empty):
                df_curr_futures = df_futures[df_futures['Date'] == curr_date.strftime("%d-%b-%y")]
#             F_t = float(df_curr_futures['Close'].tolist()[0])
            
            # Change to Settlement price
            F_t = float(df_curr_futures['Settle Price'].tolist()[0])
            # The following is the implied returns (r_f - q)
            r = float(1 / dt) * np.log(F_t / s_t)
            r_f = get_risk_free_rate_from_exact_date(curr_date)

            list_dict_vol_smile = gen_brent_iv_surface(price_lists_tenorwise, s_t, strike_lists_tenorwise, r, r_f, dt_list, option_type=prod_type, iv_list=[0.00000000000001, 50, 50])
            dict_vol_smile = gen_iv_surf_dataframe(list_dict_vol_smile)
            dict_vol_smile['Date'] = [curr_date.strftime("%d-%b-%Y") for i in range(0, len(dict_vol_smile['Moneyness']))]
            fixed_expiry_volsmiles_list.append(pd.DataFrame(dict_vol_smile))
            curr_date = next_weekday(curr_date)
            curr_date = next_working_day_incl_input_date(curr_date, holidays_list)
        df_fixed_expiry_vol_surface = pd.concat(fixed_expiry_volsmiles_list, axis=0)
        df_fixed_expiry_vol_surface.sort_values(by=['Date', 'Moneyness'])
        mthly_list_of_vol_smiles["Daily_vol_smile_" + prod_type +"_" + expiry_date.strftime("%d-%b-%Y")] = df_fixed_expiry_vol_surface
    for key in mthly_list_of_vol_smiles.keys():
        df = pd.DataFrame(mthly_list_of_vol_smiles[key])
        df = df[df['Impl_Vol'] > 0.00001]
        df.to_csv(output_path + "A1_" + stock_ident + "_" + key + ".csv", index = False)
    return mthly_list_of_vol_smiles 



In [4]:

#Generate Daily Vol smile - Time axis for each date 
#correspond to (Expiry date of monthly option - Current date) with ACT/365 Day count convention
# gen_daily_vol_smiles_for_fixed_expiries(periods_of_interest, holidays_list=holidays_list, input_path=input_data_path, output_path=output_data_path, stock_ident=stock_ident, prod_type = "CE")
gen_daily_vol_smiles_for_fixed_expiries(periods_of_interest, holidays_list=holidays_list, input_path=input_data_path, output_path=output_data_path, stock_ident=stock_ident, prod_type = "PE")


BSMVol0_price - mkt_price, BSMVol1_price - mkt_price:  25.705988440431156 9424.770728168103
Trade_terms: s_t, k, dt, option_type, iv_list: ( 9490.1 13000 0.07671232876712329 PE [1e-14, 50, 50] )
Mkt Price:  3535.4
IV0 BSM price:  3561.1059884404312
IV1 BSM price:  12960.170728168103
The initialisations doesn't contain the solution within for interpolation
BSMVol0_price - mkt_price, BSMVol1_price - mkt_price:  1.9335687148206944 9400.998308446358
Trade_terms: s_t, k, dt, option_type, iv_list: ( 9490.1 11000 0.07671232876712329 PE [1e-14, 50, 50] )
Mkt Price:  1565.3
IV0 BSM price:  1567.2335687148206
IV1 BSM price:  10966.298308446358
The initialisations doesn't contain the solution within for interpolation
BSMVol0_price - mkt_price, BSMVol1_price - mkt_price:  4.68873241409824 9884.059286068914
Trade_terms: s_t, k, dt, option_type, iv_list: ( 9902.0 11500 0.038356164383561646 PE [1e-14, 50, 50] )
Mkt Price:  1598.3
IV0 BSM price:  1602.9887324140982
IV1 BSM price:  11482.359286068913
T

{'Daily_vol_smile_PE_25-Jun-2020':     Moneyness         T     Spot  Strike       Impl_Vol         Date
 20   0.526865  0.076712   9490.1    5000   8.714540e-01  28-May-2020
 0    0.632238  0.076712   9490.1    6000   6.824839e-01  28-May-2020
 1    0.684924  0.076712   9490.1    6500   5.892324e-01  28-May-2020
 2    0.737611  0.076712   9490.1    7000   5.257227e-01  28-May-2020
 9    0.790297  0.076712   9490.1    7500   4.578607e-01  28-May-2020
 ..        ...       ...      ...     ...            ...          ...
 33   1.028597  0.002740  10305.3   10600   2.946700e-01  24-Jun-2020
 34   1.038301  0.002740  10305.3   10700 -8.302964e-300  24-Jun-2020
 35   1.048004  0.002740  10305.3   10800   3.541951e-01  24-Jun-2020
 36   1.057708  0.002740  10305.3   10900  6.185987e-300  24-Jun-2020
 37   1.067412  0.002740  10305.3   11000  6.288125e-300  24-Jun-2020
 
 [839 rows x 6 columns],
 'Daily_vol_smile_PE_30-Jul-2020':     Moneyness        T      Spot  Strike  Impl_Vol         Date


In [1]:
# expiry_date = date(2020,3,26)
# expiries_of_interest = get_all_monthly_option_expiries(periods_of_interest, holidays_list)
# dict_wkly_expiries_each_month = find_wkly_expries(expiries_of_interest, holidays_list)
# wkly_expiries_list = dict_wkly_expiries_each_month[expiry_date.strftime("%d-%b-%Y")]
# print(wkly_expiries_list)